In [1]:
#%matplotlib inline
import sys
#append the path to the folder containing bass (which should prob. contain a __init__.py file.) to pythonpath
bass_folder_loc = '/Users/morganfine-morris/Documents/Repos/bass'
sys.path.append(bass_folder_loc) 
import bass
import pandas as pd
import matplotlib.pyplot as plt

BASS ready!


QT distance, Q to Q dist

In [2]:
import os
import os.path as path
dir_content = os.listdir(bass_folder_loc)
list(enumerate(dir_content))

[(0, '.git'),
 (1, '.ipynb_checkpoints'),
 (2, '__init__.py'),
 (3, '__init__.pyc'),
 (4, 'Bass - Flowchart.jpg'),
 (5, 'Bass - Flowchart.odp'),
 (6, 'Bass - Flowchart.pdf'),
 (7, 'bass.py'),
 (8, 'bass.pyc'),
 (9, 'Change Log'),
 (10, 'Kitchen Sink-Bass.ipynb'),
 (11, 'licence.txt'),
 (12, 'pyeeg.py'),
 (13, 'pyeeg.pyc'),
 (14, 'rat34_ECG.txt'),
 (15, 'rat34_Settings.csv'),
 (16, 'README.md'),
 (17, 'Single Wave- Basic.ipynb'),
 (18, 'Single Wave- Interactive.ipynb'),
 (19, 'Single Wave-Interactive Protocol.txt')]

In [3]:
dir_content[14],dir_content[15]

('rat34_ECG.txt', 'rat34_Settings.csv')

In [15]:
Data = {}
Results = {}
Settings = {}
for i in open(path.join(bass_folder_loc,dir_content[15])):
    stripped =  i.strip()
    elements = stripped.split("\n")
    for elem in elements:
        if 'Settings' in elem:
            continue
        name, val = elem.split('\t')
        try:
            val = int(val)
        except:
            try:
                val=float(val)
            except:
                pass
            
        Settings[name] = val
        
Settings['plots folder'] = 'output'
Settings['Output Folder'] = 'output'
Settings['folder'] = bass_folder_loc
Settings['Threshold'] = 0.13
Settings['Savitzky-Golay Window Size']=251
Settings['Savitzky-Golay Polynomial'] = 6
pd.Series(Settings)

Absolute Value                                                                     True
Bandpass Highcut                                                                   none
Bandpass Lowcut                                                                    none
Bandpass Polynomial                                                                none
Baseline Type                                                                    static
Burst Area                                                                        False
Delta                                                                              0.25
Exclude Edges                                                                      True
File Type                                                                         Plain
Generate Graphs                                                                   False
Graph LCpro events                                                                False
Inter-event interval minimum (se

In [16]:
Data, Settings = bass.load_wrapper({},Settings)
Data['original'].tail()
#Data['original'][0:.3].plot()
#plt.show()
Data, Settings = bass.transform_wrapper(Data, Settings)
#Data['trans'] = Data['original']
#Data['trans'].plot(xlim=(0,.3))
Data['trans'][0:10].plot(xlim=(.3,.6))
plt.show()

Made plots folder
Data Loaded
Sampling Rate= 0.00025 (sec/frame)
rat34_ECG.txt is 606.94975 seconds long.


In [8]:
plt.show()

In [48]:
Data, Settings, Results = bass.baseline_wrapper(Data, Settings, Results)
#print Results.keys()

In [17]:
Results = bass.event_peakdet_wrapper(Data, Settings, Results)
Results = bass.event_burstdet_wrapper(Data, Settings, Results)

Mean1 has no peaks.
No Peaks Found in any time series.
No Valleys Found in any time series.
Mean1has no bursts.
No Bursts Found in any time series.


In [18]:
b=1.4
t=1.9
Data_clipped = {'original':Data['original'][b:t], 'trans':Data['trans'][b:t]}
bass.graph_ts(Data_clipped, Settings, Results)

In [ ]:
Q: find burst start, find valley immediately before it


In [17]:
Results.keys()

['Peaks-Master',
 'Peaks',
 'Bursts',
 'Bursts-Master',
 'Valleys',
 'Valleys-Master']

In [22]:
peaks_df = Results['Peaks']

In [21]:
valleys_df = Results['Valleys']

In [18]:
Results['Bursts']

{'Mean1':            Burst Start  Burst End  Burst Duration Edge Event  \
 0.13300        0.13300    0.13675         0.00375      False   
 0.30475        0.30475    0.30875         0.00400      False   
 0.47675        0.47675    0.49200         0.01525      False   
 0.64925        0.64925    0.65300         0.00375      False   
 0.82100        0.82100    0.82500         0.00400      False   
 0.99300        0.99300    1.00875         0.01575      False   
 1.16525        1.16525    1.16900         0.00375      False   
 1.33775        1.33775    1.34175         0.00400      False   
 1.50975        1.50975    1.52575         0.01600      False   
 1.68225        1.68225    1.68600         0.00375      False   
 1.85425        1.85425    1.85825         0.00400      False   
 2.02675        2.02675    2.04225         0.01550      False   
 2.19950        2.19950    2.20325         0.00375      False   
 2.37200        2.37200    2.37600         0.00400      False   
 2.54450        

In [15]:
bass.graph_ts??

In [ ]:
Results['Peaks-Master']
Results['Bursts-Master']

#Save Master Summary Files
burst_grouped = Results['Bursts-Master'].groupby(level=0)
burst_grouped = burst_grouped.describe()
burst_grouped
peak_grouped = Results['Peaks-Master'].groupby(level=0)
peak_grouped = peak_grouped.describe()
peak_grouped

In [57]:
Results['Peaks-Master']

Peaks Amplitude  Intervals
      Time                                 
Mean1 0.13475              1.410    0.17225
      0.30700              1.514    0.17200
      0.47900              1.404    0.17225
      0.65125              1.419    0.17200
      0.82325              1.506    0.17200
      0.99525              1.396    0.17200
      1.16725              1.405    0.17250
      1.33975              1.547    0.17200
      1.51175              1.397    0.17250
      1.68425              1.396    0.17225
      1.85650              1.492    0.17225
      2.02875              1.386    0.17275
      2.20150              1.421    0.17250
      2.37400              1.524    0.17250
      2.54650              1.392    0.17250
      2.71900              1.424    0.17250
      2.89150              1.488    0.17275
      3.06425              1.385    0.17275
      3.23700              1.473    0.17275
      3.40975              1.472    0.17275
      3.58250              1.369    0.17275
      3.75525              1.479    0.17275
      3.92800              1.435    0.17325
      4.10125              1.424    0.17250
      4.27375              1.500    0.17300
      4.44675              1.390    0.17300
      4.61975              1.435    0.17275
      4.79250              1.475    0.17300
      4.96550              1.371    0.17350
      5.13900              1.508    0.17275
...                          ...        ...
      601.29125            1.653    0.19225
      601.48350            1.735    0.19250
      601.67600            1.700    0.19250
      601.86850            1.799    0.19250
      602.06100            1.583    0.19250
      602.25350            1.732    0.19250
      602.44600            1.651    0.19225
      602.63825            1.684    0.19250
      602.83075            1.690    0.19225
      603.02300            1.717    0.19275
      603.21575            1.740    0.19250
      603.40825            1.598    0.19250
      603.60075            1.742    0.19250
      603.79325            1.624    0.19275
      603.98600            1.746    0.19250
      604.17850            1.639    0.19300
      604.37150            1.721    0.19300
      604.56450            1.691    0.19275
      604.75725            1.636    0.19275
      604.95000            1.733    0.19225
      605.14225            1.617    0.19250
      605.33475            1.738    0.19225
      605.52700            1.670    0.19300
      605.72000            1.756    0.19250
      605.91250            1.625    0.19225
      606.10475            1.656    0.19275
      606.29750            1.708    0.19225
      606.48975            1.635    0.19250
      606.68225            1.725    0.19250
      606.87475            1.667        NaN

[3224 rows x 2 columns]

In [58]:
Results['Bursts-Master']

Burst Start  Burst End  Burst Duration Edge Event  \
Mean1 0.13300        0.13300    0.13675         0.00375      False   
      0.30475        0.30475    0.30875         0.00400      False   
      0.47675        0.47675    0.49200         0.01525      False   
      0.64925        0.64925    0.65300         0.00375      False   
      0.82100        0.82100    0.82500         0.00400      False   
      0.99300        0.99300    1.00875         0.01575      False   
      1.16525        1.16525    1.16900         0.00375      False   
      1.33775        1.33775    1.34175         0.00400      False   
      1.50975        1.50975    1.52575         0.01600      False   
      1.68225        1.68225    1.68600         0.00375      False   
      1.85425        1.85425    1.85825         0.00400      False   
      2.02675        2.02675    2.04225         0.01550      False   
      2.19950        2.19950    2.20325         0.00375      False   
      2.37200        2.37200    2.37600         0.00400      False   
      2.54450        2.54450    2.55950         0.01500      False   
      2.71700        2.71700    2.72100         0.00400      False   
      2.88950        2.88950    2.89350         0.00400      False   
      3.06225        3.06225    3.07675         0.01450      False   
      3.23500        3.23500    3.23875         0.00375      False   
      3.40750        3.40750    3.42475         0.01725      False   
      3.58075        3.58075    3.58450         0.00375      False   
      3.75325        3.75325    3.75725         0.00400      False   
      3.92600        3.92600    3.94275         0.01675      False   
      4.09925        4.09925    4.10300         0.00375      False   
      4.27150        4.27150    4.27550         0.00400      False   
      4.44475        4.44475    4.45975         0.01500      False   
      4.61775        4.61775    4.62150         0.00375      False   
      4.79025        4.79025    4.80775         0.01750      False   
      4.96350        4.96350    4.97750         0.01400      False   
      5.13675        5.13675    5.14075         0.00400      False   
...                      ...        ...             ...        ...   
      601.28875    601.28875  601.30575         0.01700      False   
      601.48100    601.48100  601.48550         0.00450      False   
      601.67350    601.67350  601.69000         0.01650      False   
      601.86600    601.86600  601.88425         0.01825      False   
      602.05875    602.05875  602.07350         0.01475      False   
      602.25100    602.25100  602.26925         0.01825      False   
      602.44350    602.44350  602.45800         0.01450      False   
      602.63600    602.63600  602.65375         0.01775      False   
      602.82850    602.82850  602.83275         0.00425      False   
      603.02075    603.02075  603.03850         0.01775      False   
      603.21350    603.21350  603.21775         0.00425      False   
      603.40575    603.40575  603.42225         0.01650      False   
      603.59850    603.59850  603.61650         0.01800      False   
      603.79100    603.79100  603.80625         0.01525      False   
      603.98350    603.98350  604.00200         0.01850      False   
      604.17625    604.17625  604.18775         0.01150      False   
      604.36900    604.36900  604.38700         0.01800      False   
      604.56225    604.56225  604.56650         0.00425      False   
      604.75500    604.75500  604.77150         0.01650      False   
      604.94750    604.94750  604.95175         0.00425      False   
      605.14000    605.14000  605.15525         0.01525      False   
      605.33225    605.33225  605.35050         0.01825      False   
      605.52450    605.52450  605.54000         0.01550      False   
      605.71750    605.71750  605.73575         0.01825      False   
      605.91000    605.91000  605.91425         0.00425      False   
      606.10250    606.1025

In [56]:
bass.analyze??